# Tworzymy system zarządzania pamięcią dla NaszGPT

```
db/
├── current.json
├── conversations/
│   ├── 1.json
│   ├── 2.json
│   └── ...
```

tutaj utrzymy strukturę gdzie główny katalog nazwiemy `db_prototyp`

In [ ]:
from pathlib import Path
import json

In [ ]:
DB_PATH = Path("db_prototyp")
DB_CONVERSATIONS_PATH = DB_PATH / "conversations"

## Jak upewnić się, że wszystkie katalogi istnieją?

In [ ]:
if not DB_PATH.exists():
    DB_PATH.mkdir()
    DB_CONVERSATIONS_PATH.mkdir()

## Jaką strukturę chcemy zapisywać w naszej bazie danych?

```json
{
    "id": 1,
    "name": "Nazwa rozmowy",
    "chatbot_personality": "osobość chatbota",
    "messages": [
        {
            "role": "user",
            "content": "Cześć",
        },
        {
            "role": "assistant",
            "content": "Cześć, jak mogę Ci pomóc?",
            "usage": {
                "completion_tokens": 13,
                "prompt_tokens": 23,
                "total_tokens": 36                
            }
        }
    ]
}
```

## Jak jest domyślna wartość dla nowo utworzonej rozmowy?

```json
{
    "id": 1,
    "name": "Konwersacja 1",
    "chatbot_personality": "osobość chatbota...",
    "messages": []
}
```

## Jak upewnić się, że istnieje początkowa rozmowa i jest ona aktywna?

Chcemy aby po stracie NaszGPT było utworzone:

```
db/
├── current.json
├── conversations/
│   └── 1.json
│    ...
```

In [ ]:
DEFAULT_PERSONALITY = """
Jesteś pomocnikiem, który odpowiada na wszystkie pytania użytkownika.
Odpowiadaj na pytania w sposób zwięzły i zrozumiały.
""".strip()

if not DB_PATH.exists():
    DB_PATH.mkdir()
    DB_CONVERSATIONS_PATH.mkdir()
    conversation_id = 1
    conversation_name = "Konwersacja 1"

    # tworzymy nową konwersację
    with open(DB_CONVERSATIONS_PATH / f"{conversation_id}.json", "w") as f:
        f.write(json.dumps({
            "id": conversation_id,
            "name": conversation_name,
            "chatbot_personality": DEFAULT_PERSONALITY,
            "messages": [],
        }))

    # która od razu staje się aktualną
    with open(DB_PATH / "current.json", "w") as f:
        f.write(json.dumps({
            "current_conversation_id": conversation_id,
        }))

## Jak wczytać istniejące rozmową?

In [ ]:
# stwórzmy jakąś prostą konwersację i ustawmy ją jako aktualną
with open(DB_CONVERSATIONS_PATH / "13.json", "w") as f:
    f.write(json.dumps({
        "id": 13,
        "name": "Konwersacja 13",
        "chatbot_personality": DEFAULT_PERSONALITY,
        "messages": [
            {
                "author": "user",
                "content": "Cześć! Chciałbym się dowiedzieć, jakie są zasady gry w szachy.",
            },
            {
                "author": "assistant",
                "content": "Zasady gry w szachy są bardzo proste. Czy chcesz, żebym je opisał?",
                "usage": {
                    "completion_tokens": 13,
                    "prompt_tokens": 23,
                    "total_tokens": 36
                }
            },
        ],
    }))

# ustawmy ją jako aktualną
with open(DB_PATH / "current.json", "w") as f:
    f.write(json.dumps({
        "current_conversation_id": 13,
    }))

teraz naszym celem jest wczytanie aktualnej rozmowy i jej historii

In [ ]:
with open(DB_PATH / "current.json", "r") as f:
    data = json.loads(f.read())
    conversation_id = data["current_conversation_id"]

with open(DB_CONVERSATIONS_PATH / f"{conversation_id}.json", "r") as f:
    conversation = json.loads(f.read())

print(conversation_id, conversation["name"])

## Budujemy funkcję która potrafi zbudować wszystkie potrzebne struktury na początku

In [ ]:
DEFAULT_PERSONALITY = """
Jesteś pomocnikiem, który odpowiada na wszystkie pytania użytkownika.
Odpowiadaj na pytania w sposób zwięzły i zrozumiały.
""".strip()


def load_current_conversation():
    if not DB_PATH.exists():
        DB_PATH.mkdir()
        DB_CONVERSATIONS_PATH.mkdir()
        conversation_id = 1
        conversation = {
            "id": conversation_id,
            "name": "Konwersacja 1",
            "chatbot_personality": DEFAULT_PERSONALITY,
            "messages": [],
        }

        # tworzymy nową konwersację
        with open(DB_CONVERSATIONS_PATH / f"{conversation_id}.json", "w") as f:
            f.write(json.dumps(conversation))

        # która od razu staje się aktualną
        with open(DB_PATH / "current.json", "w") as f:
            f.write(json.dumps({
                "current_conversation_id": conversation_id,
            }))

    else:
        # sprawdzamy, która konwersacja jest aktualna
        with open(DB_PATH / "current.json", "r") as f:
            data = json.loads(f.read())
            conversation_id = data["current_conversation_id"]

        # wczytujemy konwersację
        with open(DB_CONVERSATIONS_PATH / f"{conversation_id}.json", "r") as f:
            conversation = json.loads(f.read())


## Jak dodać nową konwersację?

Najpierw musimy znaleźć sposób na obliczanie kolejnego ID

In [ ]:
# poszukajmy ID dla naszej kolejnej konwersacji
conversation_ids = []
for p in DB_CONVERSATIONS_PATH.glob("*.json"):
    conversation_ids.append(int(p.stem))

# conversation_ids zawiera wszystkie ID konwersacji
# następna konwersacja będzie miała ID o 1 większe niż największe
conversation_id = max(conversation_ids) + 1


kontynuujemy nową konwersację

In [ ]:
conversation = {
    "id": conversation_id,
    "name": f"Konwersacja {conversation_id}",
    "chatbot_personality": DEFAULT_PERSONALITY,
    "messages": [],
}

zapisujemy ją do pliku

In [ ]:
# tworzymy nową konwersację
with open(DB_CONVERSATIONS_PATH / f"{conversation_id}.json", "w") as f:
    f.write(json.dumps(conversation))

i aktualizujemy current.json

In [ ]:
# która od razu staje się aktualną
with open(DB_PATH / "current.json", "w") as f:
    f.write(json.dumps({
        "current_conversation_id": conversation_id,
    }))

## Jak podmieniać aktualną konwersację?

Powiedzmy, że chcemy podmienić aktualną konwersację na konwersację z ID 2

In [ ]:
conversation_id = 2

wczytujemy konwersację z ID 2

In [ ]:
with open(DB_CONVERSATIONS_PATH / f"{conversation_id}.json", "r") as f:
    conversation = json.loads(f.read())

ustawiamy ją jako aktualną

In [ ]:
with open(DB_PATH / "current.json", "w") as f:
    f.write(json.dumps({
        "current_conversation_id": conversation_id,
    }))

## Jak wylistować wszystkie konwersacje?

In [ ]:
conversations = []
for p in DB_CONVERSATIONS_PATH.glob("*.json"):
    with open(p, "r") as f:
        conversation = json.loads(f.read())
        conversations.append({
            "id": conversation["id"],
            "name": conversation["name"],
        })

conversations

## Jak zmodifikować istniejącą konwersację?

In [ ]:
conversation_id = 2
new_conversation_name = "Nowa nazwa konwersacji"

with open(DB_CONVERSATIONS_PATH / f"{conversation_id}.json", "r") as f:
    conversation = json.loads(f.read())

with open(DB_CONVERSATIONS_PATH / f"{conversation_id}.json", "w") as f:
    f.write(json.dumps({
        **conversation,
        "name": new_conversation_name,
    }))